In [4]:
#########
# START: COPIED FROM <https://github.com/PlusLabNLP/Sonnet-Gen>
# This code was NOT refactored as I was training the model for the exact same task as the author's of the github repo. 
# I did do some fine tuning which I have highlighted in the code.
##########


In [ ]:
# Importing libraries
import json
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

In [5]:
!nvidia-smi

Wed Dec 13 02:09:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda:0' if cuda.is_available() else 'cpu'

In [7]:
class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

In [8]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 500 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [9]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    inputs = []
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            

            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=512, 
                min_length = 250,
              num_beams = 5,
              no_repeat_ngram_size = 5,
              #topp = 0.9,
              #do_sample=True,
              repetition_penalty=5.8, 
              length_penalty=1, 
              early_stopping=True
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            input_text = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in ids]
            if _%50==0:
                outputs = model(
                        input_ids=ids,
                        attention_mask=mask,
                        decoder_input_ids=y_ids,
                        labels=lm_labels,
                        )
                loss = outputs[0]
                console.print(f'Completed {_}')
                console.print('loss: '+ str(loss))
            
            predictions.extend(preds)
            actuals.extend(target)
            inputs.extend(input_text)
    return inputs, predictions, actuals


In [10]:
def generate(tokenizer, model, device, loader):
    model.eval()
    inputs = []
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            

            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=512, 
                min_length = 100,
              num_beams = 4,
              no_repeat_ngram_size = 5,
              #topp = 0.9,
              #do_sample=True,
              repetition_penalty=5.8, 
              length_penalty=1, 
              early_stopping=True
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            input_text = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in ids]
            if _%50==0:
                console.print(f'Completed {_}')

            predictions.extend(preds)
            inputs.extend(input_text)
    return inputs, predictions

In [11]:
def BartTrainer(
    dataframe, source_text, target_text, model_params, model, tokenizer, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So x% of the data will be used for training and the rest for validation.
    train_size = 0.998
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = YourDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    ) # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)
        console.log(f"[Initiating Validation]...\n")
        inputs, predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Input': inputs, "Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions"+str(epoch)+".csv"))

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [12]:
# read data in
f = open('news_yake_keywords.json', errors='ignore').readlines()
news =  json.loads(f[0])
g = open('roc_yake_keywords.json', errors='ignore').readlines()
roc =  json.loads(g[0])
h = open('poetry_yake_keywords-2.json', errors='ignore').readlines()
poetry =  json.loads(g[0])
all_scary =  json.loads(f[0])

In [13]:
len(news), len(roc), len(poetry)

(4501, 6501, 6501)

In [14]:
all_poetry_foundation = roc + poetry + news

In [15]:
len(all_poetry_foundation)

17503

In [16]:
# Generate title with prompt and masked keywords to input into model
special_token = '.'
mask_token = '<MASK>'
eos_token = '</s>'

X_titles = []
y_keywords = []
template = [mask_token, mask_token, mask_token]
prompt = 'Generate keywords for the title: '
title_set = []

for poem in all_poetry_foundation:
    title_set.append(poem['Theme'])
    title = prompt + poem['Theme']
    paddings = []
    temp = []
    count = 0
    for key in poem['keywords']:
        if key == ['<paragraph>']:
            continue
        count += 1
        mask = template[:len(key)]
        paddings.append('Keywords '+ str(count) + ': '+ str(mask) )        
        temp.append('Keywords '+ str(count) + ': '+ str(key))

    paddings = (" "+special_token+" ").join(paddings).replace('<paragraph> ','')
    temp = (" "+special_token+" ").join(temp).replace('<paragraph> ','')

    X_titles.append(title + '. ' + paddings)
    #X_titles.append(title)
    y_keywords.append(temp) 

In [17]:
data = [X_titles, y_keywords]
df = pd.DataFrame(np.array(data).T, columns = ['title', 'keywords'])
df.head()

,title,keywords
0,Generate keywords for the title: david drops t...,"Keywords 1: ['david', 'recently', 'noticed'] ...."
1,Generate keywords for the title: marcus buys k...,"Keywords 1: ['marcus', 'event', 'needed'] . Ke..."
2,Generate keywords for the title: melodys trip ...,"Keywords 1: ['melodys', 'aquarium', 'parents']..."
3,Generate keywords for the title: my first girl...,"Keywords 1: ['internet', 'girlfriend', 'met'] ..."
4,Generate keywords for the title: new hair colo...,"Keywords 1: ['andy', 'party', 'invited'] . Key..."


In [18]:
df['title'][10005]

"Generate keywords for the title: a hundred bucks. Keywords 1: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 2: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 3: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 4: ['<MASK>', '<MASK>'] . Keywords 5: ['<MASK>', '<MASK>', '<MASK>']"

In [19]:
df['keywords'][10005]

"Keywords 1: ['jim', 'bucks', 'needed'] . Keywords 2: ['opportunity', 'decided', 'scour'] . Keywords 3: ['bucks', 'job', 'paid'] . Keywords 4: ['jobsite', 'drove'] . Keywords 5: ['bucks', 'posed', 'ten']"

In [20]:
# HYPERPARAMETER TUNING: I HAVE EXPERIMENTED WITH BATCHSIZE, LEARNING RATE, MODEL SIZE. THE PARAMETERS BELOW ARE THE BEST PARAMETERS
model_params = {
    "TASK" : "0503-mix",
    "MODEL": "facebook/bart-large",  
    "TRAIN_BATCH_SIZE": 2,  # training batch size
    "VALID_BATCH_SIZE": 2,  # validation batch size
    "TRAIN_EPOCHS": 4,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 5e-6,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 512,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [21]:
# tokenzier for encoding the text
tokenizer = BartTokenizer.from_pretrained(model_params["MODEL"])

In [22]:
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = BartForConditionalGeneration.from_pretrained(model_params["MODEL"])
model = model.to(device)

In [23]:
tokens = tokenizer(df['title'][15005])

len(tokens['input_ids'])

93

In [24]:
output_dir= model_params['MODEL']+"_batch_"+ str(model_params['TRAIN_BATCH_SIZE']) + "_lr_"+ str(model_params['LEARNING_RATE'])+ model_params['TASK']
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [25]:
BartTrainer(
    dataframe=df,
    source_text="title",
    target_text="keywords",
    model_params=model_params,
    model = model,
    tokenizer = tokenizer,
    output_dir = output_dir
)

[02:10:01] [Model]: Loading facebook/bart-large...                                                 ]8;id=322826;file:///var/tmp/ipykernel_195673/3558859256.py\3558859256.py]8;;\:]8;id=348417;file:///var/tmp/ipykernel_195673/3558859256.py#16\16]8;;\
                                                                                                                   

           [Data]: Reading data...                                                                 ]8;id=7309;file:///var/tmp/ipykernel_195673/3558859256.py\3558859256.py]8;;\:]8;id=85355;file:///var/tmp/ipykernel_195673/3558859256.py#21\21]8;;\
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|   Generate keywords for the title: david drops the     | Keywords 1: ['david', 'recently', 'noticed'] . Keywords|
| weight. Keywords 1: ['<MASK>', '<MASK>', '<MASK>'] .   |    2: ['reason', 'examined', 'habits'] . Keywords 3:   |
| Keywords 2: ['<MASK>', '<MASK>', '<MASK>'] . Keywords  |  ['realized', 'hed', 'eating'] . Keywords 4: ['diet',  |
|    3: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 4:     | 'stopped', 'burger'] . Keywords 5: ['weeks', 'started',|
|['<MASK>', '<MASK>', '<MASK>'] . Keywords 5: ['<MASK>', |                         'feel']                        |
|                  '<MASK>', '<MASK>']                   |                                                        |
| Generate keywords for the title: marcus buys khakis.   | Keywords 1: ['marcus', 'event', 'needed'] . Keywords 2:|
| Keywords 1: ['<MASK>', '<MASK>', '<MASK>'] . Keywords  |      ['casual', 'clothes', 'formal'] . Keywords 3:     |
|    2: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 3:     |       ['khakis', 'decided', 'buy'] . Keywords 4:       |
|['<MASK>', '<MASK>', '<MASK>'] . Keywords 4: ['<MASK>', |      ['perfectly', 'pair', 'bought'] . Keywords 5:     |
|'<MASK>', '<MASK>'] . Keywords 5: ['<MASK>', '<MASK>',  |              ['marcus', 'event', 'happy']              |
|                       '<MASK>']                        |                                                        |
+-----------------------------------------------------------------------------------------------------------------+

FULL Dataset: (17503, 2)

TRAIN Dataset: (17468, 2)

...

[11:03:48] [Saving Model]...                                                                       ]8;id=564811;file:///var/tmp/ipykernel_195673/3558859256.py\3558859256.py]8;;\:]8;id=643363;file:///var/tmp/ipykernel_195673/3558859256.py#85\85]8;;\
                                                                                                                   

[11:03:51] [Validation Completed.]                                                                 ]8;id=226938;file:///var/tmp/ipykernel_195673/3558859256.py\3558859256.py]8;;\:]8;id=981537;file:///var/tmp/ipykernel_195673/3558859256.py#96\96]8;;\
                                                                                                                   

[Model] Model saved @ facebook/bart-large_batch_2_lr_5e-060503-mix/model_files

[Validation] Generation on Validation data saved @ facebook/bart-large_batch_2_lr_5e-060503-mix/predictions.csv

[Logs] Logs saved @ facebook/bart-large_batch_2_lr_5e-060503-mix/logs.txt

In [ ]:
#########
# END: COPIED FROM <https://github.com/PlusLabNLP/Sonnet-Gen>
# This code was NOT refactored as I was training the model for the exact same task as the author's of the github repo. 
# I did do some fine tuning which I have highlighted in the code.
##########